In [1]:
from langchain_chroma import Chroma
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.prompts import PromptTemplate
from langchain_google_genai.llms import GoogleGenerativeAI
from langchain.chains import LLMChain
from api_key import GOOGLE_API_KEY

c:\Users\Aneel\OneDrive\Projects\Python-projects\Medbot\medbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = {'max_new_tokens': 512, 'temperature': 0.8}
llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY, config=config)


In [3]:
pdf = "docs\medical_book.pdf"
loader = PyPDFLoader(pdf)
loaded_data = loader.load()
print(len(loaded_data))

759


In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=200)
chunks = text_splitter.split_documents(loaded_data)
print (f"Your document has been split into {len(chunks)} chunks")


Your document has been split into 1235 chunks


In [5]:
vectordb = Chroma.from_documents(documents=chunks, embedding=FastEmbedEmbeddings(), persist_directory="db_CC")
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=vectordb.as_retriever(search_kwargs={"k": 3}))

Fetching 5 files: 100%|██████████| 5/5 [00:00<?, ?it/s]


In [6]:
relevant_docs = compression_retriever.base_retriever.get_relevant_documents("What do people think is a flaw of capitalism?")
print (f"You have {len(relevant_docs)} relevant docs")
for i in relevant_docs:
    print (i.page_content + '\n\n')

c:\Users\Aneel\OneDrive\Projects\Python-projects\Medbot\medbot\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


You have 3 relevant docs
organisms. It seems that the enzymes occurring in fresh
fruit and vegetables encourage these beneficial organ-
isms. One of the results of eating processed denatured
foods is that this natural balance is upset, and food may
begin to rot in the bowel instead of being processed.
Decomposing matter can cause a toxic condition and
may lead to many health problems, as constipation caus-
es backed up pollution of the body cells. The process of
repair and elimination of wastes enters a downward spi-
ral which at best will cause fatigue, lack of energy and
GALE ENCYCLOPEDIA OF MEDICINE 2 855Colonic irrigation


Failure to thrive may also result from lack of avail-
able food or the quality of the food offered. This can be
due to economic factors in the family, parental beliefs and
concepts of nutrition , or neglect of the child. In addition,
if the baby is being breast fed, the quality or quantity of
the mother’s milk may be the source of the problem.
Psychosocial probl

In [7]:
question = "How to treat corneal ulcers?"
compressed_docs = compression_retriever.get_relevant_documents(question)


In [8]:
prompt_template = """
Use the given information context to give appropriate answer for the user's question.
If you don't know the answer, just say that you know the answer, but don't make up an answer.
Context: {context}
Question: {question}
Only return the appropriate answer and nothing else.
Helpful answer:
"""

prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [9]:
chain = LLMChain(llm=llm, prompt=prompt, verbose=True)
output = chain.run(context=compressed_docs, question=question)
print(output)


c:\Users\Aneel\OneDrive\Projects\Python-projects\Medbot\medbot\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
c:\Users\Aneel\OneDrive\Projects\Python-projects\Medbot\medbot\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:

Use the given information context to give appropriate answer for the user's question.
If you don't know the answer, just say that you know the answer, but don't make up an answer.
Context: [Document(page_content='A corneal ulcer needs to be treated aggressively, as it\ncan result in loss of vision. The first step is to eliminate\ninfection. Broad spectrum antibiotics will be used\nbefore the lab results come back. Medications may then\nbe changed to more specifically target the cause of the\ninfection. A combination of medications may be neces-\nsary. Patients should return for their follow-up visits so\nthat the doctor can monitor the healing process. The\ncornea can heal from many insults, but if it remains\nscarred, corneal transplantation may be necessary to\nrestore vision. If the corneal ulcer is large, hospitaliza-\ntion may be necessary.', metadata={'page': 310, 'source': 'docs\\medical_book.pdf'})]
Question: How to t